# `reader_layout.py`
**Contains 5 reader layout functions**
- `create_reader_layout()`
- `show_current_view(state, layout)`
- `summary_view_panel(state, layout)`
- `transaction_view_panel(state, layout)`
- `account_view_panel(state, layout)`

In [ ]:
from rich.layout import Layout
from rich.table import Table
from rich.panel import Panel
from rich.layout import Layout
from rich.align import Align
from rich import box
from rich.columns import Columns
from rich.text import Text

In [ ]:

# Use common layout components(create_header, create_footer) fom common_layouts.py

def create_reader_layout():
    """
    Creates the base layout for the reader process
    return: Layout - A Rich Layout object with header, body, and footer sections
    """
    # create the main layout
    layout = Layout()

    #split the layout into header, body, and footer
    layout.split_column(
        Layout(name="header", size=3),
        Layout(name="body", size=16),
        Layout(name="footer", size=3)
    )

    header_text = "💸  BUDGET TRACKER CLI 💸"
    color = "magenta"
    footer_text = "Monitoring: shared_state.json | Use writer.py to send commands | Press Ctrl+C to stop the reader process"

    # .update() method can be used to refresh or change layout components
    # it comes from rich library's Layout class
    layout["header"].update(create_header(header_text, color))
    layout["footer"].update(create_footer(footer_text))

    return layout
#================================================================================================

# From common_layouts.py
#================================================================================================
def create_header(header_text, color):
    """
    Create the header section
    param header_text: str - The text to display in the header
    param color: str - The color style for the header text and border
    return: Panel - A Rich Panel object containing the header
    """
    header_text = Text(header_text, style=f"bold {color}", justify="center")
    return Panel(header_text, box=box.DOUBLE, border_style=color)

def create_footer(footer_text):
    """
    Create the footer section
    param footer_text: str - The text to display in the footer
    return: Panel - A Rich Panel object containing the footer
    """
    footer_text = Text(
        footer_text,
        style="dim cyan",
        justify="center"
    )
    return Panel(footer_text, box=box.SIMPLE)
#================================================================================================

In [ ]:
# From reader_layouts.py
#================================================================================================
def show_current_view(state, layout):
    """
    Update the reader's body based on the current view in state
    param state: dict - The current state dictionary
    param layout: Layout - The Rich Layout object to update
    return: layout - The updated Rich Layout object
    """

    current_view = state.get("current_view", "SUMMARY")
    if current_view == "SUMMARY":
        return summary_view_panel(state, layout)
    elif current_view == "ACCOUNTS":
        return account_view_panel(state, layout)
    elif current_view == "TRANSACTIONS":
        return transaction_view_panel(state, layout)
    else:
        return layout

def summary_view_panel(state, layout):
    """
    Update the reader's body with summary view content based on the current state
    param state: dict - The current state dictionary
    param layout: Layout - The Rich Layout object to update
    return: layout - The updated Rich Layout object
    """

    # get the current view from state to display in the title
    current_view = state.get("current_view", None)

    # get all accounts from state to access each account and their transactions
    accounts = state.get("accounts", [])

    # account table with all accounts
    all_accounts_table = create_account_table(accounts)

    # transaction table with all transactions from all accounts
    transactions = [trans for account in accounts for trans in account.get("transactions", [])]
    transactions_table = create_transaction_table(transactions)

    # add both tables side by side in a columns panel
    tables_panel = Columns([all_accounts_table, transactions_table])
    body = Panel(
        Align.center(tables_panel, vertical="middle"),
        title=current_view,
        border_style="green",
        box=box.ROUNDED
    )
    return layout["body"].update(body)

# this gets all transactions from all accounts
def transaction_view_panel(state, layout):
    """
    Update the reader's body with summary view content based on the current state
    param state: dict - The current state dictionary
    param layout: Layout - The Rich Layout object to update
    return: layout - The updated Rich Layout object
    """

    # get the current view from state to display in the title
    current_view = state.get("current_view", None)

    # get all accounts from state to access their transactions
    accounts = state.get("accounts", [])

    # get all transactions from all accounts
    transactions = [trans for account in accounts for trans in account.get("transactions", [])]

    # create a table for all transactions
    transactions_table = create_transaction_table(transactions)

    # create a panel with the transactions table
    # tables_panel = Columns([transactions_table])
    body = Panel(
        Align.center(transactions_table, vertical="middle"),
        title=current_view,
        border_style="green",
        box=box.ROUNDED
    )
    return layout["body"].update(body)

def account_view_panel(state, layout):
    """
    Update the reader's body with summary view content based on the current state
    param state: dict - The current state dictionary
    param layout: Layout - The Rich Layout object to update
    return: layout - The updated Rich Layout object
    """

    current_view = state.get("current_view", None)
    accounts = state.get("accounts", [])
    all_accounts_table = create_account_table(accounts)
    tables_panel = Columns([all_accounts_table])
    body = Panel(
        Align.center(tables_panel, vertical="middle"),
        title=current_view,
        border_style="green",
        box=box.ROUNDED
    )
    return layout["body"].update(body)

#================================================================================================


# From accounts_layouts.py
#================================================================================================

def create_account_table(accounts):
    """
    Create a Rich Table to display account information
    param accounts: list - A list of account dictionaries
    return: Table - A Rich Table object containing account information
    """
    if not accounts:
        content = Text("No accounts available.", justify="center", style="white")
        return content
    else:
        table = Table(title="Accounts Overview", box=box.SIMPLE_HEAVY)
        table.add_column("Account Name", style="cyan", no_wrap=True)
        table.add_column("Account Type", style="magenta")
        table.add_column("Balance", justify="right", style="green")

        for account in accounts:
            table.add_row(
                account.get("name", "Unnamed"),
                account.get("type", "N/A"),
                f"${account.get('balance', 0):.2f}"
            )
    return table


# From transactions_layouts.py
#================================================================================================

def create_transaction_table(transactions):
    """
    Create a Rich Table to display transaction information
    param transactions: list - A list of transaction dictionaries
    return: Table - A Rich Table object containing transaction information
    """
    if not transactions:
        content = Text("No transactions available.", justify="center", style="white")
        return content
    else:
        table = Table(title="Transactions", box=box.SIMPLE_HEAVY)
        table.add_column("Amount", justify="right", style="green")
        table.add_column("Description", style="yellow")

        for transaction in transactions:
            table.add_row(
                f"${transaction.get('amount', 0):.2f}",
                transaction.get("description", "No description")
            )
    return table


#================================================================================================